In [65]:
import numpy as np 
import pandas as pd 

from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import cross_validate, KFold, RandomizedSearchCV
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier

In [37]:
breast = pd.read_csv('data.csv')
breast.drop(['id','Unnamed: 32'],axis=1,inplace=True)
breast['diagnosis'] = breast['diagnosis'].map({'M':1,'B':0})

In [38]:
# Separando os dados e atributos e target
x = breast.drop(['diagnosis'],axis=1).values
y = breast.diagnosis.values

# Normalizando os dados
x_std = StandardScaler().fit_transform(x)

In [39]:
svm = SVC(probability=True)
knn = KNeighborsClassifier()
tree = DecisionTreeClassifier()
na_ba = GaussianNB()

## Aplicando o Vote Classifier

In [40]:
# voting com hard
vote_cla = VotingClassifier(estimators=[('svm',svm),('tree',tree),('naive',na_ba)]) 

In [41]:
# colocando o kfold
kfold = KFold(n_splits=7)

In [42]:
nome_metricas = ['accuracy', 'precision', 'recall', 'f1']

metricas_ran = cross_validate(vote_cla,x_std, y, cv=kfold, scoring=nome_metricas)

for met in metricas_ran:
  print(f'-{met}')
  print(f"-- {metricas_ran[met]}")
  media = np.mean(metricas_ran[met])
  desvio = np.std(metricas_ran[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

-fit_time
-- [0.39824986 0.31847644 0.34601879 0.31095719 0.47130585 0.30975389
 0.35481191]
Média do fit_time: 0.35851056235177176
Desvio 0.054396664051577115
Intervalo [0.250,0.467]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.05977511 0.15354109 0.0546484  0.06690431 0.05218291 0.0323298
 0.09670925]
Média do score_time: 0.07372726712908063
Desvio 0.03718975638041978
Intervalo [-0.001,0.148]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.93902439 0.90243902 0.98765432 0.96296296 0.98765432 0.97530864
 0.92592593]
Média do test_accuracy: 0.9544242267819502
Desvio 0.030400461744997
Intervalo [0.894,1.015]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision
-- [0.96363636 0.89285714 1.         1.         1.         0.94117647
 0.82608696]
Média do test_precision: 0.9462509905147831
Desvio 0.06127825068157313
Intervalo [0.824,1.069]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-

In [43]:
vote_cla2 = VotingClassifier(estimators=[('knn',knn),('tree',tree),('naive',na_ba)]) 

In [44]:
metricas_ran2 = cross_validate(vote_cla2,x_std, y, cv=kfold, scoring=nome_metricas)

for met in metricas_ran2:
  print(f'-{met}')
  print(f"-- {metricas_ran2[met]}")
  media = np.mean(metricas_ran2[met])
  desvio = np.std(metricas_ran2[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

-fit_time
-- [0.1631186  0.11901832 0.04779983 0.12104392 0.07830644 0.22413874
 0.06487346]
Média do fit_time: 0.11689989907400948
Desvio 0.056856992878076926
Intervalo [0.003,0.231]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.20894027 0.03831077 0.07826281 0.137537   0.10728335 0.12586713
 0.10446   ]
Média do score_time: 0.11438018935067314
Desvio 0.04906205092901355
Intervalo [0.016,0.213]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.92682927 0.90243902 0.98765432 0.95061728 0.98765432 0.98765432
 0.92592593]
Média do test_accuracy: 0.9526820665032046
Desvio 0.03291126726916002
Intervalo [0.887,1.019]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision
-- [0.96296296 0.92307692 1.         0.96       1.         1.
 0.82608696]
Média do test_precision: 0.9531609775088036
Desvio 0.05828181580188295
Intervalo [0.837,1.070]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_r

In [45]:
vote_cla3 = VotingClassifier(estimators=[('knn',knn),('svm',svm),('naive',na_ba)],voting='soft') 

In [46]:
metricas_ran3 = cross_validate(vote_cla3,x_std, y, cv=kfold, scoring=nome_metricas)

for met in metricas_ran3:
  print(f'-{met}')
  print(f"-- {metricas_ran3[met]}")
  media = np.mean(metricas_ran3[met])
  desvio = np.std(metricas_ran3[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

-fit_time
-- [0.21832752 0.21198082 0.17141533 0.28300071 0.30253506 0.1980412
 0.18725109]
Média do fit_time: 0.2246502467564174
Desvio 0.045688394657715625
Intervalo [0.133,0.316]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.01716065 0.05559468 0.13187122 0.01685786 0.13932371 0.08805251
 0.02124715]
Média do score_time: 0.06715825625828334
Desvio 0.049430509398469824
Intervalo [-0.032,0.166]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.93902439 0.8902439  0.97530864 0.96296296 0.98765432 0.98765432
 0.95061728]
Média do test_accuracy: 0.9562094033638748
Desvio 0.03177150780913091
Intervalo [0.893,1.020]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision
-- [0.96363636 0.83870968 1.         1.         1.         1.
 0.9047619 ]
Média do test_precision: 0.9581582779739463
Desvio 0.05873921673391507
Intervalo [0.841,1.076]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_r

## Aplicando Random Forest

In [47]:
rf = RandomForestClassifier(criterion='entropy')

In [48]:
metricas_ran4 = cross_validate(rf,x_std, y, cv=kfold, scoring=nome_metricas)

for met in metricas_ran4:
  print(f'-{met}')
  print(f"-- {metricas_ran4[met]}")
  media = np.mean(metricas_ran4[met])
  desvio = np.std(metricas_ran4[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

-fit_time
-- [2.23531318 1.11118317 1.90746713 2.12098575 2.20555973 1.61590052
 1.8004384 ]
Média do fit_time: 1.8566925525665283
Desvio 0.3692502855086326
Intervalo [1.118,2.595]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.06967902 0.20309711 0.20334244 0.09402442 0.23837423 0.13927341
 0.20125914]
Média do score_time: 0.16414996555873326
Desvio 0.05904049052217021
Intervalo [0.046,0.282]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.91463415 0.91463415 1.         0.96296296 0.96296296 0.97530864
 0.95061728]
Média do test_accuracy: 0.9544457349335397
Desvio 0.02888430248908643
Intervalo [0.897,1.012]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision
-- [0.98039216 0.92592593 1.         1.         0.91666667 0.94117647
 0.9047619 ]
Média do test_precision: 0.9527033035436397
Desvio 0.03720126790103625
Intervalo [0.878,1.027]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-t

## Votinclassifier colocando regressão logistica

In [53]:
rg = LogisticRegression()

vote_cla4 = VotingClassifier(estimators=[('knn',knn),('tree',tree),('rg',rg)])

metricas_ran5 = cross_validate(vote_cla4,x_std, y, cv=kfold, scoring=nome_metricas)

for met in metricas_ran5:
  print(f'-{met}')
  print(f"-- {metricas_ran5[met]}")
  media = np.mean(metricas_ran5[met])
  desvio = np.std(metricas_ran5[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

-fit_time
-- [0.49478269 0.3161273  0.32265592 0.13714433 0.23138738 0.18610549
 0.3294754 ]
Média do fit_time: 0.28823978560311453
Desvio 0.10876972910820473
Intervalo [0.071,0.506]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.33671904 0.080724   0.2445116  0.05008411 0.05631709 0.20728612
 0.11407042]
Média do score_time: 0.1556731973375593
Desvio 0.10120884764365966
Intervalo [-0.047,0.358]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.95121951 0.98780488 0.97530864 0.96296296 0.97530864 0.98765432
 0.97530864]
Média do test_accuracy: 0.9736525143029208
Desvio 0.012083485531037932
Intervalo [0.949,0.998]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision
-- [0.96428571 1.         1.         1.         0.95652174 1.
 0.95238095]
Média do test_precision: 0.9818840579710145
Desvio 0.02116645452221098
Intervalo [0.940,1.024]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_r

## Bagging com regressão logistica

In [54]:
bag_reg = BaggingClassifier(base_estimator=rg)

metricas_ran6 = cross_validate(bag_reg,x_std, y, cv=kfold, scoring=nome_metricas)

for met in metricas_ran6:
  print(f'-{met}')
  print(f"-- {metricas_ran6[met]}")
  media = np.mean(metricas_ran6[met])
  desvio = np.std(metricas_ran6[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

-fit_time
-- [1.32919312 0.61799288 0.58069825 0.48354173 0.54833388 0.48871279
 0.70177531]
Média do fit_time: 0.6786068507603237
Desvio 0.2746791937493012
Intervalo [0.129,1.228]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.05518723 0.01581311 0.01419091 0.01776314 0.03116202 0.01244068
 0.01418042]
Média do score_time: 0.02296250207083566
Desvio 0.014385441728352013
Intervalo [-0.006,0.052]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.97560976 0.98780488 0.96296296 0.97530864 1.         0.98765432
 0.98765432]
Média do test_accuracy: 0.9824278401514174
Desvio 0.011137941839590456
Intervalo [0.960,1.005]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision
-- [1.         1.         1.         1.         1.         0.94444444
 0.95454545]
Média do test_precision: 0.9855699855699857
Desvio 0.022975012017446835
Intervalo [0.940,1.032]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*

## Adaboost com regressão logistica

In [56]:
ranges = np.arange(0.1,1.1,0.1)
for i in ranges:
    ada_rg = AdaBoostClassifier(base_estimator=rg,learning_rate=i)
    metricas_ran7 = cross_validate(ada_rg,x_std, y, cv=kfold, scoring=nome_metricas)
    print(f'Learning Rate: {i}')
    for met in metricas_ran7:
        print(f'-{met}')
        print(f"-- {metricas_ran7[met]}")
        media = np.mean(metricas_ran7[met])
        desvio = np.std(metricas_ran7[met])
        print(f'Média do {met}: {media}')
        print(f'Desvio {desvio}')
        print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
        print('-*-'*20)

Learning Rate: 0.1
-fit_time
-- [2.11972785 1.16945815 1.38446403 1.45567012 1.21408725 1.18748474
 1.23046684]
Média do fit_time: 1.394479853766305
Desvio 0.3123077064967422
Intervalo [0.770,2.019]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.06643605 0.05874109 0.09748077 0.06388164 0.07214427 0.05212522
 0.10316443]
Média do score_time: 0.07342478207179479
Desvio 0.018023689307370545
Intervalo [0.037,0.109]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.93902439 0.97560976 0.96296296 0.96296296 1.         0.98765432
 0.97530864]
Média do test_accuracy: 0.9719318621757644
Desvio 0.01814123020183395
Intervalo [0.936,1.008]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision
-- [1.         1.         1.         1.         1.         0.94444444
 0.95238095]
Média do test_precision: 0.9852607709750567
Desvio 0.02340109687137223
Intervalo [0.938,1.032]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*

In [61]:
parametros = {'penalty':['l1','l2','elasticnet','none'],'C':np.arange(1,101,1),'solver':['liblinear',
            'saga','sag','newton-cg','lbfgs'],'multi_class':['auto','ovr','multinomial']}

rand_sear = RandomizedSearchCV(rg,param_distributions=parametros).fit(x_std,y)


/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/scipy/optimiz

In [62]:
rand_sear.best_params_

{'solver': 'saga', 'penalty': 'l1', 'multi_class': 'multinomial', 'C': 25}

In [63]:
rgh = LogisticRegression(solver='saga',C=25,penalty='l1',multi_class='multinomial')

In [64]:
metricas_ran8 = cross_validate(rgh,x_std, y, cv=kfold, scoring=nome_metricas)
for met in metricas_ran8:
        print(f'-{met}')
        print(f"-- {metricas_ran8[met]}")
        media = np.mean(metricas_ran8[met])
        desvio = np.std(metricas_ran8[met])
        print(f'Média do {met}: {media}')
        print(f'Desvio {desvio}')
        print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
        print('-*-'*20)

/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linea

-fit_time
-- [0.25356793 0.10856748 0.13407612 0.35350347 0.25164127 0.31209898
 0.2054379 ]
Média do fit_time: 0.23127044950212752
Desvio 0.08242432734416386
Intervalo [0.066,0.396]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.00764728 0.01559782 0.00908041 0.02305317 0.01193595 0.00937033
 0.02012157]
Média do score_time: 0.013829503740583147
Desvio 0.0055037226322452604
Intervalo [0.003,0.025]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.96341463 0.97560976 0.96296296 0.97530864 0.98765432 0.98765432
 0.98765432]
Média do test_accuracy: 0.977179851163591
Desvio 0.010189008687722944
Intervalo [0.957,0.998]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision
-- [0.98181818 1.         1.         1.         0.95833333 0.94444444
 0.95454545]
Média do test_precision: 0.977020202020202
Desvio 0.02243316190925285
Intervalo [0.932,1.022]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*

/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
